In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import csv
import time
import scipy.stats
#from sklearn import tree

In [2]:
#read in data
use = [1,1,1,1]
all_data = []
all_data2 = []



#read in blob dataset

X_training = []
y_training = []
header = []
c = 0

with open("DataSets/Wind Power Generation Data - Forecasting/Location1.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        if( c == 0):
            header = row[1:-1]
            c+=1
            continue
        X_training.append([float(x) for x in row[1:-1]])
        y_training.append(float(row[-1]))
        #print(row)

X_train, X_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.2) # 80% training and 20% testing data

train = [(x,y) for x,y in zip (X_train,y_train)]
test = [(x,y) for x,y in zip (X_test,y_test)]

all_data2.append([train,test])
print(len(X_training))
print(len(y_training))
all_data.append([X_train, X_test, y_train, y_test,header])


X_training = []
y_training = []
header = []
c = 0
with open("DataSets/auto+mpg/auto-mpg.data","r") as file:
    r = csv.reader(file)
    header  = ['cylinders','displacement','horsepower','weight','acceleration','model year','origin']
    for row in r:   
        line = row[0].split(' ')
        #print(line)      
        if( '?' in line):

            continue

        buff = [float(x) for x in row[0][1:].split(' ') if x!='' and x.replace('.', '', 1).isdigit() ]
        X_training.append([float(x) for x in row[0][1:].split(' ') if x!='' and x.replace('.', '', 1).isdigit() ][0:7])
        y_training.append(float(row[0][0]))
        #print(row)

X_train, X_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.2) # 80% training and 20% testing data

train = [(x,y) for x,y in zip (X_train,y_train)]
test = [(x,y) for x,y in zip (X_test,y_test)]

all_data2.append([train,test])
print(len(X_training))
print(len(y_training))
all_data.append([X_train, X_test, y_train, y_test,header])



X_training = []
y_training = []
header = []
c = 0

with open("DataSets/Graduate Admission 2/Admission_Predict_Ver1.1.csv","r") as file:
    r = csv.reader(file)
    
    for row in r:
        if(c == 0):
            header = row[1:-1]
            c+=1
            continue
        X_training.append([float(x) for x in row[1:-1]])
        y_training.append(float(row[-1]))


X_train, X_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.2) # 80% training and 20% testing data

train = [(x,y) for x,y in zip (X_train,y_train)]
test = [(x,y) for x,y in zip (X_test,y_test)]

all_data2.append([train,test])
print(len(X_training))
print(len(y_training))

all_data.append([X_train, X_test, y_train, y_test,header])


#read in house price dataset
X_training = []
y_training = []
header = []
c = 0
with open("DataSets/house_att.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        if(c == 0):
            header = row
            c+=1
            continue
        X_training.append([float(x) for x in row])
        
with open("DataSets/house_price.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        y_training.append(float(row[0]))
  
print(len(X_training))
print(len(y_training))
X_train, X_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.2) # 80% training and 20% testing data

train = [(x,y) for x,y in zip (X_train,y_train)]
test = [(x,y) for x,y in zip (X_test,y_test)]

all_data2.append([train,test])

all_data.append([X_train, X_test, y_train, y_test,header])
print(len(all_data2))

43800
43800
392
392
500
500
10000
10000
4


In [3]:
class Node:
    def __init__(self,x,y,top,bottom,mse,mean,depth,att):
        self.x = x
        self.y = y
        self.top = top
        self.bottom = bottom
        self.sub_nodes = []
        self.mse = mse
        self.mean = mean
        self.depth = depth
        self.att = att #attribute index
        #self.predict = mean
        self.leaf = 0


In [4]:
import numpy as np
class PushTree:
    def __init__(self,x,y,step_num,ratio,limit,L=1):
        self.x = x #features
        self.y = y #labels
        
        self.step_size = 0
        self.step_num = step_num
        self.ratio = ratio
        self.leaf = 0
        #print(y)
        self.mean = np.array(y).mean()

        self.limit = limit

        self.L = L
        self.att = 0
        self.sub_nodes = []
        self.depth = []

    def mse(self,data):
        if(len(data) == 0):

            return float('NaN')
            #print(data,'Here')
        return np.square(np.subtract( data,data.mean() )).mean()

    def sort_f(self,X,y,column):

        sortf = zip(X,y)

        sortf= sorted(sortf,key = lambda x:x[0][column])
        #print("itt")
        X = np.array([x for x,y in sortf])
        y = np.array([y for x,y in sortf])
    
        return X,y
    
    def generate_mask(self,x):
        mask = []# ne index mask legyen hanem a split pontok
        #print("Max: ",str(max(x)))
        step_size = (max(x)-min(x))/self.ratio#self.ratio = len(child nodes)
        self.step_size = step_size

        for i in range(0,self.ratio+1):
            mask.append(min(x) + (i*step_size) )
        return mask
    




    def idenctical(self,a,b):
        
        for i in range(len(a)):
            if(a[i]!=b[i]):
                return 0
        return 1
    
    def globalError(self,mask,vector,x,y):
        error_sum_local=0
        split_data_local=[]
        for i in range(1,len(mask)):
                    
                    split_data_local.append([x[ (mask[i-1] <= vector) & (vector<= mask[i])], y[ (mask[i-1]<=vector) & (vector<=mask[i]) ]])

                    error_sum_local += self.mse(split_data_local[-1][1])
        return error_sum_local
    
    def getPartitions(self,vector,y,mask,j,d1,d2):
        sub_att_left_rshift =   vector[(mask[j-1] <= vector) & (vector < mask[j]+d1)]#bal
        sub_lab_left_rshift =  y[(mask[j-1] <= vector) & (vector < mask[j]+d1)] 


        if(j == len(mask)-2):

            sub_att_right_rshift =  vector[(mask[j]+d1 <= vector) & (vector <= mask[j+1])]
            sub_lab_right_rshift =  y[(mask[j]+d1 <= vector) & (vector <= mask[j+1])]

            sub_att_right_lshift =   vector[(mask[j]-d2 <= vector) & (vector <= mask[j+1])]#jobb
            sub_lab_right_lshift =  y[(mask[j]-d2 <= vector) & (vector <= mask[j+1])] 

        else:
            sub_att_right_rshift =  vector[(mask[j]+d1 <= vector) & (vector < mask[j+1])]
            sub_lab_right_rshift =  y[(mask[j]+d1 <= vector) & (vector < mask[j+1])] 

            sub_att_right_lshift =   vector[(mask[j]-d2 <= vector) & (vector < mask[j+1])]#jobb
            sub_lab_right_lshift =  y[(mask[j]-d2 <= vector) & (vector < mask[j+1])] 

        sub_att_left_lshift =   vector[(mask[j-1] <= vector) & (vector < mask[j]-d2)]#bal
        sub_lab_left_lshift =  y[(mask[j-1] <= vector) & (vector < mask[j]-d2)] 
        
        return sub_lab_left_rshift, sub_lab_right_rshift, sub_lab_right_lshift,sub_lab_left_lshift

    def getLoss(self,sub_lab_left,sub_lab_right,sub_lab_left_rshift,sub_lab_right_rshift,sub_lab_right_lshift,sub_lab_left_lshift):
                #print('Right,left: ',len(sub_lab_left),len(sub_lab_right))
            h0 = len(sub_lab_left)+len(sub_lab_right)
            #h0=1
            #error = (len(sub_lab_left)/h0*self.mse(sub_lab_left) + len(sub_lab_right)/h0*self.mse(sub_lab_right) )

            sub_lab = np.concatenate((sub_lab_left,sub_lab_right),axis=0)
            error = self.mse(sub_lab)
            h = len(sub_lab_left_rshift) + len(sub_lab_right_rshift)
            h2 = len(sub_lab_right_lshift) + len(sub_lab_left_lshift)

            e1 = (len(sub_lab_right_rshift)/h*self.mse(sub_lab_right_rshift) +  len(sub_lab_left_rshift)/h*self.mse(sub_lab_left_rshift))
            e2 = (len(sub_lab_right_lshift)/h2*self.mse(sub_lab_right_lshift) +  len(sub_lab_left_lshift)/h2*self.mse(sub_lab_left_lshift))

            loss_right_shift = error -  (len(sub_lab_right_rshift)/h*self.mse(sub_lab_right_rshift) +  len(sub_lab_left_rshift)/h*self.mse(sub_lab_left_rshift)) #jobb

            loss_left_shift =  error - (len(sub_lab_right_lshift)/h2*self.mse(sub_lab_right_lshift) +  len(sub_lab_left_lshift)/h2*self.mse(sub_lab_left_lshift)) #bal

            return loss_right_shift,loss_left_shift,e1,e2





    
    def calculate_split_std(self,vector,y,column,split_value):
    #calculate the standard deviation on the splited data,add them together and return that value
    
    
        variance_sum = 0
        #get the splited dataset
        sub_labels = [ y[vector<=split_value] ] 
        sub_labels.append( y[vector>split_value])

        #print(sub_labels[0])
        #print(sub_labels)
        for i in sub_labels:

            #if(len(i) == 0):         
            #    return -1,0#!!!!!!!!!!!!!!!!!!!4444444

            variance_sum += (len(i)/len(y))*self.mse(i)
            
        variance = self.mse(y) - variance_sum
    
        return variance,sub_labels


    def binary_split(self,x,y):

        min_sub_labels = []
        split_value = -1
        att = -1
        re_data = []

        x = np.array(x)
        y = np.array(y)
        #print(len(x))
        lowest_v = -1
        for k in range(len(x[0])-0):
           
            V = 0
            
            #print(k)
            x,y = self.sort_f(x,y,k)
            vector = x[:,k] # get an attribute
            
            for i in range(0,len(vector)-1,2):
                svalue = (vector[i]+vector[i+1])/2

                V,sub_labels = self.calculate_split_std(vector,y,k,svalue)

                if(lowest_v == -1):
                        
                        re_data = []

                        re_data.append([x[ vector<= svalue],y[ (vector<=svalue)]])
                        re_data.append([x[ svalue<vector],y[ (svalue<vector) ]])
                        #print(re_data)

                        lowest_v = V
                        split_value = svalue
                        att = k
                        min_sub_labels = sub_labels                                         

                if(V<lowest_v and V != -1):
                        split_value = svalue
                        lowest_v = V
                        att = k
                        re_data = []
                

    

                        re_data.append([x[ vector<= svalue],y[ (vector<=svalue)]])
                        re_data.append([x[ svalue<vector],y[ (svalue<vector) ]])
                        #print(re_data)
                        min_sub_labels = sub_labels
                    
        re_mask = [min(vector),split_value,max(vector)]

        return re_data,att,re_mask
        
    

    def best_split(self,x,y,depth):#check all possible boundaries based on the given constraints


        split_data = []
        min_error = 99999999999999999999999999
        re_data = []#legyen egyenlo egy felosztásaal mindenképpen
        re_mask = 0
        att = -1
        x = np.array(x)
        y = np.array(y)
        
        if(len(x) < 30):
            re_data,att,re_mask = self.binary_split(x,y)

            return re_data,att,re_mask
        

        for k in range(len(x[0])-0):
           
            #print('///////////////////////////////////')
            x,y = self.sort_f(x,y,k)
            split_vector = []
            split_data = []
            vector = x[:,k] # get an attribute
            
            mask = self.generate_mask(vector)

            error_sum = 0
            #print(mask)
            if(mask.count(mask[0]) == len(mask)):
                #in case we have categorical like features

                continue


            #print(k,'---------------------------------------------------------------------------------------------------------------')
            length = len(mask)
            optimal = [0 for x in range(self.ratio-1)]
            local_min_error = 9999999999999999999999999
            
            pre_masks = []
            
            while(0 in optimal):
                           
                sub_attributes = [  vector[(mask[i] < vector) & (vector<=mask[i+1])] for i in range(0,len(mask)-1)] 
                sub_labels = [ y[(mask[i]<vector) & (vector<=mask[i+1])] for i in range(0,len(mask)-1)]
                error = 0
                split_data_local = []
                error_sum_local = 0

                j = 1
                
                while(j<=length-2):
                    error_sum_local_left = 0
                    error_sum_local_right = 0
                    
                    if(optimal[j-1] == 1):
                        j+=1
                        continue
                    
                    d1  =  abs(max(vector)-min(vector))/200
                    d2  =  abs(max(vector)-min(vector))/200
                    #constraints check
                    if(mask[j]+d1>=mask[j+1]):
                        d1 = 0
                    if(mask[j]-d2<=mask[j-1]):
                        d2 = 0


                    sub_lab_left = y[(mask[j-1] <= vector) & (vector < mask[j])] 

                    if(j+1 == len(mask)-1):
                        sub_lab_right = y[(mask[j] <= vector) & (vector <= mask[j+1])]
                    else:
                        sub_lab_right = y[(mask[j] <= vector) & (vector < mask[j+1])]
                    
                    if( (len(sub_lab_left) == 0 or len(sub_lab_right) == 0) and len(mask)>3 and j!=0 and j!=length-1):
                        
                        mask.remove(mask[j])
                       
                        length -= 1
                        if(j>1):
                            j-=1
                        sub_lab_left = y[(mask[j] <= vector) & (vector < mask[j+1])] 
                        sub_lab_right = y[(mask[j-1] <= vector) & (vector <= mask[j])]
                        
                    
                    sub_lab_left_rshift, sub_lab_right_rshift, sub_lab_right_lshift,sub_lab_left_lshift = self.getPartitions(vector,y,mask,j,d1,d2)
                    
                    loss_right_shift,loss_left_shift,e1,e2 = self.getLoss(sub_lab_left,sub_lab_right,
                                                                    sub_lab_left_rshift,
                                                                    sub_lab_right_rshift,
                                                                    sub_lab_right_lshift,
                                                                    sub_lab_left_lshift)

                   
                    if( loss_right_shift>loss_left_shift and error>e1 and mask[j]+d1<mask[j+1]):
                        mask[j]+=d1 
                
                    elif(loss_right_shift<loss_left_shift and error>e2 and mask[j]-d2>mask[j-1] ):
                        mask[j]-=d2
                        
                    else:
                        optimal[j-1] = 1
                    j+=1

                
                
                if(len(pre_masks)<2):
                    pre_masks.append(mask.copy())
                    continue
    
                if(self.idenctical(pre_masks[-2],mask)):
                        #print(pre_masks,mask)
                        err1 = self.globalError(mask,vector,x,y)
                        err2 = self.globalError(pre_masks[-1],vector,x,y)
                        if(err1<=err2):
                            #print("break")
                            for i in range(len(optimal)):
                                optimal[i] = 1
                            break
                        else:
                            mask = pre_masks[-1].copy()                          
                            break

                pre_masks.append(mask.copy())

                   
            for i in range(1,len(mask)):
                split_vector.append([vector[ (mask[i-1] <= vector) & (vector<=mask[i]) ], y[ (mask[i-1]<=vector) &  (vector<=mask[i] )] ])
                split_data.append([x[ (mask[i-1] <= vector) & (vector<= mask[i])], y[ (mask[i-1]<=vector) & (vector<=mask[i]) ]])

                error_sum += self.mse(split_data[-1][1])

            if(error_sum<min_error):
                min_error = error_sum
                re_data = split_data 
                re_mask = mask
                att = k

        return re_data,att,re_mask
   

    def fit(self,x,y,depth=0,root=0):

        split_data,att,mask = self.best_split(x,y,depth)

        sub_nodes = []
        s = 0
        for i,node in enumerate(split_data):

            sub_node = Node(
                x = np.array(node[0]),
                y = np.array(node[1]),
                top = mask[i+1],#max(node[0][:,att]),
                bottom = mask[i],#min(node[0][:,att]),
                mse = np.square(np.subtract( np.array(node[1]),np.array(node[1]).mean() )).mean(),
                mean = np.array(node[1]).mean(),
                depth = depth+1,
                att = att
            )

            s += len(node[0])
            #print(len(node[0]))
            sub_nodes.append(sub_node)#minden sub_node a fa subnodjába megy

        #print(self)
        for node in sub_nodes:

            if(len(node.x)<self.limit or self.mse(node.y)<=0.0):#slef.limit

                self.depth.append(depth+1)
                node.leaf = 1
            else: 
                #print(depth)
                node.sub_nodes = self.fit(node.x,node.y,depth+1)
        if(root):
            self.sub_nodes = sub_nodes
            self.att = att
        else:
            return sub_nodes

        

    def predict(self,X,node, root=0):

        if(root == 0 and node.leaf == 1):

            return node.mean
        else:
            found = False
            #print('Value: ',X[node.att])
            for child in node.sub_nodes:

                if(X[child.att] >= child.bottom and X[child.att] <= child.top):
                   
                    return self.predict(X,child)
        
            min_dist = 999999999999999999999
            min_child = 0
            for child in node.sub_nodes:
                dist1 = abs(X[child.att]-child.bottom)
                dist2 = abs(X[child.att]-child.top)
                if(min(dist1,dist2)<min_dist):
                    min_dist = min(dist1,dist2)
                    min_child = child
                
                    
            return self.predict(X,min_child)
        return node.mean

In [6]:
forest = []
times = []
depth = []
limits = [120,115,110,105,100,95,90,85,80,75,70,65,60,55,50,45,40,35,30,25,20,15,10]
#limits = [10]


for i in all_data:

    f = []
    t = []
    d = []
    for l in limits:
        
        print("-------------------------------------------------------")
        print(l)
        start = time.time()
        # create a regressor object
        tree = PushTree(i[0],i[2],10,3,l)#irirs 30,3;10,3 blob 20,3;10,5;30,5
        PushTree.fit(tree,i[0],i[2],root=1)
    
        end = time.time()
        f.append(tree)
        t.append(end-start)
        d.append(tree.depth)
        
        
    forest.append(f)
    times.append(t)
    depth.append(d)

-------------------------------------------------------
120
-------------------------------------------------------
115
-------------------------------------------------------
110
-------------------------------------------------------
105
-------------------------------------------------------
100
-------------------------------------------------------
95
-------------------------------------------------------
90
-------------------------------------------------------
85
-------------------------------------------------------
80
-------------------------------------------------------
75
-------------------------------------------------------
70
-------------------------------------------------------
65
-------------------------------------------------------
60
-------------------------------------------------------
55
-------------------------------------------------------
50
-------------------------------------------------------
45
---------------------------------------------------

In [7]:
predicted_value = []#predicted points
difference = []
MSE = []
sum_y = []

all_mse = []
print(len(forest))
#for each tree we calculate the mean squared error
#all_data = [X_train,X_test,y_train,y_test,header]
for v in range(len(forest)):
    MSE = []
    print(len(forest[v]))
    
    for i in forest[v]:
        
        z=v
        '''if(v == 2):
            z+=1'''
        j = all_data[z]
        
        X_test = j[1]
        y_test = j[3]
        pv = []
        error = 0
        mse = 0
        for k in range(len(X_test)): 
            #print('////////////////////////////////////')
            p = [i.predict(X_test[k],i,root=1)]

    
            error += abs(y_test[k]-p[0])
            #mse += (y_test[k]-round(p[0],0))**2
            mse += (y_test[k]-p[0])**2
            pv.append(p)



        sum_y.append(sum(y_test))
        predicted_value.append(pv)
        difference.append(error)
        MSE.append(mse/len(X_test))

    all_mse.append(MSE)


#[[0.33411703274340626]]   
print(all_mse)
print(time)


4
23
23
23
23
[[0.026906347789306572, 0.026943717812445436, 0.026963037799024554, 0.027026653936761617, 0.027137622132050548, 0.02714942145734214, 0.02723763158955493, 0.027360133756255355, 0.0274173884883939, 0.027521577309804677, 0.02761388880446835, 0.027819218532988816, 0.027858818611032624, 0.02797296051082341, 0.028544985145660624, 0.02892436809786389, 0.02913438731271578, 0.02950237765238328, 0.03071324586010889, 0.030712634449054168, 0.0307118454050915, 0.03072257528804419, 0.030741265469203763], [0.27069123325092403, 0.27069123325092403, 0.27069123325092403, 0.27069123325092403, 0.27069123325092403, 0.27069123325092403, 0.1640457495167495, 0.1640457495167495, 0.16302932638931053, 0.16302932638931053, 0.16302932638931053, 0.16302932638931053, 0.16302932638931053, 0.16302932638931053, 0.16302932638931053, 0.15142017209295838, 0.12578140040988295, 0.12536599261169493, 0.12768666771718012, 0.12704949176129077, 0.12789022068458586, 0.12683309892170655, 0.1258305985310205], [0.00489

'[0.028621882976704538, 0.2928322784810126, 0.006449158179355402, 11.394822888219903]\n#divide = 4\n[0.03029302423740405, 0.2021052296915015, 0.005387932709593095, 10.78060687210206]\n[0.02940464900973105, 0.22493142285058026, 0.005387932709593095, 10.78060687210206]\n[0.028719784673477723, 0.22493142285058026, 0.005387932709593095, 10.78060687210206]\n#divide = 3\n[0.029322449040447836, 0.17297750605164708, 0.0058957809070389776, 9.944607715564873]\n[0.028034358327197542, 0.17297750605164708, 0.0058957809070389776, 9.944607715564873]'

In [8]:
with open("Results3/hybrid/difference_hybrid_push3_mse_nbrt2.csv", 'a',newline='') as f:
    # create the csv writer
    writer = csv.writer(f)

    # write a row to the csv file
    for row in all_mse:
        writer.writerow(row)
        
    #for row in all_mse:
    #    writer.writerow(row)
with open("Results3/hybrid/time_hybrid_push3_mse_nbrt2.csv", 'a',newline='') as f:
    # create the csv writer
    writer = csv.writer(f)

    for row in times:
        
        writer.writerow(row)
    writer.writerow(" ")
with open("Results3/hybrid/depth_hybrid_push3_mse_nbrt2.csv", 'a',newline='') as f:
    # create the csv writer
    writer = csv.writer(f)

    for row in depth:
        
        writer.writerow(row)
    writer.writerow(" ")

#[[0.33411703274340626]]   
print(all_mse)
print(time)

[[0.026906347789306572, 0.026943717812445436, 0.026963037799024554, 0.027026653936761617, 0.027137622132050548, 0.02714942145734214, 0.02723763158955493, 0.027360133756255355, 0.0274173884883939, 0.027521577309804677, 0.02761388880446835, 0.027819218532988816, 0.027858818611032624, 0.02797296051082341, 0.028544985145660624, 0.02892436809786389, 0.02913438731271578, 0.02950237765238328, 0.03071324586010889, 0.030712634449054168, 0.0307118454050915, 0.03072257528804419, 0.030741265469203763], [0.27069123325092403, 0.27069123325092403, 0.27069123325092403, 0.27069123325092403, 0.27069123325092403, 0.27069123325092403, 0.1640457495167495, 0.1640457495167495, 0.16302932638931053, 0.16302932638931053, 0.16302932638931053, 0.16302932638931053, 0.16302932638931053, 0.16302932638931053, 0.16302932638931053, 0.15142017209295838, 0.12578140040988295, 0.12536599261169493, 0.12768666771718012, 0.12704949176129077, 0.12789022068458586, 0.12683309892170655, 0.1258305985310205], [0.004895512291163882,